#### Arnoldi Iteration Method
``` txt
```
* [Arnoldi_iteration wikipedia](https://en.wikipedia.org/wiki/Arnoldi_iteration)

In [17]:
import numpy as np

In [18]:
def arnoldi_iteration(A, b, n: int, eps = 1e-12):
    """Compute a basis of the (n + 1)-Krylov subspace of the matrix A.

    This is the space spanned by the vectors {b, Ab, ..., A^n b}.

    Parameters
    ----------
    A : array_like
        An m × m array.
    b : array_like
        Initial vector (length m).
    n : int
        One less than the dimension of the Krylov subspace, or equivalently the *degree* of the Krylov space. Must be >= 1.
    
    Returns
    -------
    Q : numpy.array
        An m x (n + 1) array, where the columns are an orthonormal basis of the Krylov subspace.
    h : numpy.array
        An (n + 1) x n array. A on basis Q. It is upper Hessenberg.
    """
    h = np.zeros((n + 1, n))
    Q = np.zeros((A.shape[0], n + 1))
    # Normalize the input vector
    Q[:, 0] = b / np.linalg.norm(b, 2)  # Use it as the first Krylov vector
    for k in range(1, n + 1):
        v = np.dot(A, Q[:, k - 1])  # Generate a new candidate vector
        for j in range(k):  # Subtract the projections on previous vectors
            h[j, k - 1] = np.dot(Q[:, j].conj(), v)
            v = v - h[j, k - 1] * Q[:, j]
        h[k, k - 1] = np.linalg.norm(v, 2)
        if h[k, k - 1] > eps:  # Add the produced vector to the list, unless
            Q[:, k] = v / h[k, k - 1]
        else:  # If that happens, stop iterating.
            return Q, h
    return Q, h

In [19]:
n = 4
np.random.seed(42)
A = np.array([
    [5, 4, 1, 1],
    [4, 6, 2, 1],
    [1, 2, 7, 2],
    [1, 1, 2, 8]
], dtype=float)
# A = np.random.rand(n, n)


# 随机生成一个初始向量
b = np.random.rand(A.shape[0])


# 2. 运行Arnoldi迭代
Q, h = arnoldi_iteration(A, b, n)
# 3. 计算Arnoldi近似特征值 (Ritz值)
# 我们需要从上Hessenberg矩阵 h 的左上角 n x n 子矩阵中提取特征值
# 因为Arnoldi算法通常在第n步生成一个 n x n 的上Hessenberg矩阵
H_n = h[:n, :n]

print(f"A=\n{A}")
print(f"upper Hessenberg=\n{h}")
print(f"H_n= 如果 A 是 Hermitian matrices 的时候, Hessenberg matrix 会变成上三角矩阵!!!\n{H_n}")

arnoldi_eigenvalues = np.linalg.eigvals(H_n)

# 4. 使用NumPy计算真实的特征值
true_eigenvalues = np.linalg.eigvals(A)

# 5. 比较结果
print("\nArnoldi近似特征值 (Ritz值):")
a_sorted = np.sort(arnoldi_eigenvalues)[::-1]
print(a_sorted)

print("\n矩阵A的真实特征值:")
t_sorted = np.sort(true_eigenvalues)[::-1]
print(t_sorted)

print("\ndiff:")
print(np.abs(t_sorted[0:n] - a_sorted))


A=
[[5. 4. 1. 1.]
 [4. 6. 2. 1.]
 [1. 2. 7. 2.]
 [1. 1. 2. 8.]]
upper Hessenberg=
[[ 1.14853812e+01  2.33634959e+00  1.64313008e-14 -1.09245946e-13]
 [ 2.33634959e+00  2.19107211e+00  9.93121278e-01 -2.04281037e-14]
 [ 0.00000000e+00  9.93121278e-01  5.26401825e+00  6.51441978e-01]
 [ 0.00000000e+00  0.00000000e+00  6.51441978e-01  7.05952845e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.71074489e-14]]
H_n= 如果 A 是 Hermitian matrices 的时候, Hessenberg matrix 会变成上三角矩阵!!!
[[ 1.14853812e+01  2.33634959e+00  1.64313008e-14 -1.09245946e-13]
 [ 2.33634959e+00  2.19107211e+00  9.93121278e-01 -2.04281037e-14]
 [ 0.00000000e+00  9.93121278e-01  5.26401825e+00  6.51441978e-01]
 [ 0.00000000e+00  0.00000000e+00  6.51441978e-01  7.05952845e+00]]

Arnoldi近似特征值 (Ritz值):
[12.04757912  7.28665883  5.27507676  1.39068529]

矩阵A的真实特征值:
[12.04757912  7.28665883  5.27507676  1.39068529]

diff:
[7.10542736e-15 8.88178420e-15 3.55271368e-15 3.99680289e-15]


```txt
    经典的 Arnoldi 的缺陷和改进方向, 参考下面 Google Gemini 的回答.
```
---
### 隐式重启Arnoldi方法 (Implicitly Restarted Arnoldi Method)

您的 `arnoldi_iteration` 代码实现了经典的 Arnoldi 方法，它能生成一个 Krylov 子空间的正交基。然而，对于大型矩阵，这种方法有两个主要限制：

1.  **计算和存储成本高**：为了获得好的特征值近似，您可能需要很大的 `n` 值。这会使 `Q` 和 `h` 矩阵变得非常大，导致大量的计算和存储需求。
2.  **数值不稳定性**：随着 `n` 增大，`Q` 的列向量之间的正交性会逐渐丧失，影响结果的准确性。

**隐式重启Arnoldi方法（IRAM）**正是为了解决这些问题而出现的。它不是一次性构建一个巨大的 Krylov 子空间，而是在每次迭代后通过“重启”来精简这个子空间，从而达到以下改进：

#### 改进的核心思想

IRAM 的核心思想是**周期性地剔除 Krylov 子空间中那些对逼近目标特征值贡献较小的向量，并用新的、更优的向量来替换它们**。

具体来说，IRAM 改进了以下几个方面：

1.  **控制 Krylov 子空间的大小**
    * **您的代码**：迭代次数 `n` 是固定的。如果您想提高精度，就必须增大 `n`，这会无限制地增加计算和存储成本。
    * **IRAM**：IRAM 设定一个较小的固定大小 `k` 来进行 Arnoldi 迭代。一旦达到 `k` 步，它会通过一个巧妙的数学过程（QR 算法）来“重启”迭代。这个过程会保留与您感兴趣的特征值（通常是最大或最小）相关的 Ritz 向量，同时剔除那些无关的向量。

2.  **效率和可扩展性**
    * **您的代码**：每次迭代都从头开始，计算量线性增长。
    * **IRAM**：通过隐式重启，IRAM 每次只处理一个较小的 Krylov 子空间。它避免了维护一个巨大的 `Q` 和 `h` 矩阵，因此在大规模特征值问题上**计算更高效**且**内存占用更少**。

3.  **收敛性和稳定性**
    * **您的代码**：没有内置的收敛加速机制。对于某些矩阵，收敛可能很慢，甚至由于数值误差而停滞。
    * **IRAM**：重启过程本质上是一种**多项式滤波**。它有效地“过滤”掉了那些不想要的特征值，从而**加速了目标特征值的收敛**。同时，因为它始终在一个较小的正交基上工作，所以**保持了更好的数值稳定性**。

#### 总结

| 特性 | 您的经典 Arnoldi 方法 | 隐式重启Arnoldi方法 (IRAM) |
| :--- | :--- | :--- |
| **迭代大小** | 固定 `n`，随精度要求增大 | 固定较小的 `k`，通过重启保持大小 |
| **计算成本** | 随着 `n` 线性增长，成本高 | 每次迭代成本低，整体高效 |
| **内存占用** | 随 `n` 线性增长，占用大 | 始终保持在低维，占用小 |
| **收敛速度** | 较慢，可能需要很多次迭代 | 通过“过滤”不想要的特征值来加速收敛 |
| **数值稳定性** | 随着 `n` 增大，正交性可能丧失 | 始终在小基上工作，稳定性更好 |

简而言之，**IRAM 是对经典 Arnoldi 方法的一个工程和算法上的重大改进**，使其能够高效、稳定地处理大规模矩阵的稀疏特征值问题。它通过精巧的“重启”机制，在保持核心 Arnoldi 算法优点的同时，解决了其在实际应用中的计算和内存瓶颈。